## Introductory note

Prof praising SQL's Compositionality

The ability to compose complex expressions from simple queries by feeding the results of queries as inputs of other queries

this nesting of queries can happen an unbounded amount of times, leading to some more complex nesting

## Simple use of NOT IN

EX: Find all movies that are **not** currently playing

```sql
SELECT  title FROM Movie
WHERE   title NOT IN (
    SELECT  title FROM Schedule
)
```

### Q: Why can't this be flattened?

Hand-wavy "proof"

- Basic queries w/ no nestint are **monotonic**

"The answer never decreases when the database increases"

DB1 $\subseteq$ DB2 *implies* Query(DB1) $\subseteq$ Query(DB2)

- But queries using NOT IN are **not monotonic**

Using example from above (find all movies not currently playing), if Schedule increases, the answer might decrease

## More complex use of NOT IN

another use case is modular development. Can use nesting to build an intermediary relation that can be used in outer queries
\+ pipelines can be built in a toolbase fashion

EX: Find names of employees with the max salary

```sql
SELECT  name    FROM Employee
WHERE   salary  NOT IN (
    SELECT  e.salary
    FROM    Employee e, Employee f
    WHERE   e.salary < f.salary
)
```

Intuition: salary is maximum if it is NOT among salaries e.salary lower than some f.salary

basically not in the group of non-max salaries

### Another complex example

> $\exists$ "there exists" quanitifier is easily accomplished with any conditional in standard defined by SQL, however the $\forall$ "for all" quanitifier has no direct syntax in the standard. Expressing it requires going through all these lengths (contorted but it's the only way out).

General Strategy for all queries of this nature: DeMorgan's law of first order logic is used to simulate universal quanitification with existential quantification (i.e. laws of negations).

EX: Find actors playing in **every** movie by "Berto"

$$ \begin{align*}
    \forall x : p(x)    &== \neg (\neg (\forall x : p(x))) \\
                        &== \neg (\exists x : \neg p(x)) \\
\end{align*} $$

$$ \begin{align*}
    \forall m \in \text{Movies} : (\text{a acted in m}) &== \neg (\exist m \in \text{Movie} : \neg (\text{a acted in m}))
\end{align*} $$

Recall the prompt: Find actors playing in **every** movie by "Berto"

1. Violating actors (there is some Berto movie they missed)
   - 2 iterators through Movie, for actor and for director


```sql
SELECT  Actor   FROM Movie
WHERE   Actor   NOT IN (
    
    SELECt  m1.Actor
    FROM    Movie m1, Movie m2,
    WHERE   m2.Director = "Berto"
        AND NOT EXISTS (                -- equivalent to slide example but uses EXISTS instead of IN syntax
            SELECT  *   FROM Movie 3
            WHERE   m3.Actor = m1.Actor
                AND m3.Title = m2.Title
        )

)
```

> Note: Should be able to complete hw 1 now!

#### Aside
Student Q: Why don't we just grab data from DB and then post process it in another scripting language or something?

Profs A: It can be done and it IS done by some application (*brief interruption to say HUH they DO??? That sounds so scuffed lmaooo, sounds like something Justin*). Some caveats:
- bottelnecks at network transfer speeds
- ideally do most of the heavy lifting to the optimised DBMS and only work on niche/prototypy/quick changes with scripts
- whenever you can, DON'T post process